# CHILD ML Analysis
    - ML Pipeline for CHILD Dataset from the very beginning

##### Step One: Feature Selection Strategies (Insights)

##### Step Two: Making ML Pipelines

##### Step Three: Tunning & Feature Engineering/Selection (data review) to Improve Performance (Insights)

##### Step Four: Visualization of model performance

##### Step Five: Paper, Deployment

### Libraries

In [2]:
# Preferences of autoformatting & Multiple Output
%load_ext nb_black

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import warnings

warnings.filterwarnings("ignore")

import researchpy as rp  # For auto-statistics/EDA of dataframe
from tqdm.notebook import tqdm  # For process display
import importlib  # For viewing library structure (dependancies, etc)

import sys

sys.path.append("../src")

from data import *
from utils import *
from conf import *

import utils as UT
import data as DT


import random

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

<IPython.core.display.Javascript object>

### Data

In [3]:
# df_raw = generate_raw_xlsx()
df_child = load_child_with_more()

Loading ('../data/addon/Prenatal Q91PRNMH18WK.xlsx', '../output/CHILD_raw.xlsx', '../data/addon/breastfeeding data.xlsx', '../data/addon/Prenatal Q91PRNMH18WK.xlsx'), and merging
The dataframe merged with more information is saved to ../output/ with name of CHILD_with_addon.xlsx


<IPython.core.display.Javascript object>

In [4]:
df_targeted = target_selector(df_child, target_mapping={2: 1})

The dimension of original dataframe for process is (3455, 157)
Number of categorical features is:  56
Number of numeric features is:  24
Number of target variables is:  13
Number of dropped features is:  63
The difference of features of targeted and original dataframe is :{'Subject_Number'}
The number of missing value for target label is: 809
------------------------------------------------------
Note: Target variable can be one of: 
 ['Asthma_Diagnosis_3yCLA', 'Asthma_Diagnosis_5yCLA', 'Recurrent_Wheeze_1y', 'Recurrent_Wheeze_3y', 'Recurrent_Wheeze_5y', 'Wheeze_Traj_Type', 'Medicine_for_Wheeze_5yCLA', 'Viral_Asthma_3yCLA', 'Triggered_Asthma_3yCLA', 'Viral_Asthma_5yCLA', 'Triggered_Asthma_5yCLA', 'Cumulative_Wheeze_36m', 'Cumulative_Wheeze_60m']
------------------------------------------------------
****Target variable will be renamed to y for easy access.**** 



<IPython.core.display.Javascript object>

In [5]:
df_shrunk, X, y, df_dropped = sample_selector(df_targeted)

A total of 30 / 412 (7.3%) for asthma positive are dropped due to more than 10 missing value in the sample.

The total number of dropped samples is 196


<IPython.core.display.Javascript object>

In [12]:
# Restore raw data
df = df_shrunk.copy()
X = df_shrunk.drop(columns="y").copy()
y = df_shrunk["y"].copy()

<IPython.core.display.Javascript object>

###  Test Run of Transformers

In [13]:
ApgarTransformer(engineer_type="Log1p").fit_transform(X)
BirthTransformer().fit_transform(X)
Log1pTransformer().fit_transform(X)
RespiratoryTransformer(first_18m_divide=True).fit_transform(X)
DiscretizePSS().fit_transform(X)
ColumnFilter().fit_transform(X)
ImputerStrategizer().fit_transform(X)
CatNaNImputer().fit_transform(X)

,Gest_Days,Weight_0m,Weight_3m,Weight_12m,BF_Implied_Duration,PSS_36week,CSED_36week,PSS_18week,CSED_18week,PSS_6m,...,BF_Status_6m,Child_Ethinicity,Mode_of_delivery,Mother_Condition_Delivery,First_10min_Measure,Respiratory_Report_Months,Respiratory_Infections,Severity_of_Respiratoryinfections,Wheeze_3m,Noncold_Wheeze_3m
Subject_Number,,,,,,,,,,,,,,,,,,,,,
20059,276.0,3.346,6.20,NaN,12.0,10.0,14.0,13.0,7.0,16.0,...,Partial,Caucasian,1.0,"Bleeding,Nausea,Gestational Diabetes",None,"[nan, nan, nan, 3.0882956878850107, 6.24229979...","[nan, nan, nan, 'No', 'No', 'No', 'LRTI', 'LRT...","[nan, nan, nan, 'No', 'No', 'No', 'Mild', 'Mil...",0.0,0.0
20061,269.0,3.370,6.53,9.30,8.0,5.0,4.0,11.0,4.0,9.0,...,Partial,HalfCaucas,8.0,NaN,None,"[nan, nan, nan, 3.383983572895277, 6.340862422...","[nan, nan, nan, 'No', 'No', 'LRTI', 'LRTI', 'L...","[nan, nan, nan, 'No', 'No', 'Mild', 'Severe', ...",0.0,0.0
20062,279.0,3.055,5.60,8.18,6.0,15.0,15.0,10.0,3.0,18.0,...,Partial,Caucasian,1.0,Nausea,NaN,"[nan, nan, nan, nan, 2.9897330595482545, 7.129...","[nan, nan, nan, nan, 'No', 'LRTI', 'LRTI', 'LR...","[nan, nan, nan, nan, 'No', 'Mild', 'Mild', 'Mi...",0.0,0.0
20063,273.0,3.485,5.90,NaN,11.0,5.0,3.0,4.0,1.0,8.0,...,Exclusive,Caucasian,1.0,Nausea,None,"[nan, nan, nan, nan, 2.9568788501026693, 13.96...","[nan, nan, nan, nan, 'No', 'No', 'No', 'No', '...","[nan, nan, nan, nan, 'No', 'No', 'No', 'No', '...",0.0,0.0
20064,258.0,3.940,7.30,9.45,2.5,24.0,12.0,17.0,18.0,23.0,...,Zero,Caucasian,6.0,"Bleeding,Nausea,Hypertension",NaN,"[nan, 3.0882956878850107, 7.063655030800822, 1...","[nan, 'No', 'No', 'No', 'No', 'No', 'No', 'No'...","[nan, 'No', 'No', 'No', 'No', 'No', 'No', 'No'...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,280.0,3.800,5.52,8.50,14.0,26.0,39.0,12.0,8.0,12.0,...,Exclusive,Caucasian,1.0,Other,None,"[nan, nan, 2.9897330595482545, 8.0821355236139...","[nan, nan, 'No', 'No', 'No', 'LRTI', 'LRTI', '...","[nan, nan, 'No', 'No', 'No', 'Mild', 'Mild', '...",0.0,0.0
50860,281.0,3.520,5.52,8.20,0.5,8.0,5.0,5.0,0.0,9.0,...,Zero,HalfCaucas,1.0,None,None,"[nan, nan, nan, nan, nan, 2.7597535934291586, ...","[nan, nan, nan, nan, nan, 'No', 'No', 'LRTI', ...","[nan, nan, nan, nan, nan, 'No', 'No', 'Mild', ...",0.0,0.0
50862,272.0,3.648,7.04,9.70,12.0,4.0,3.0,2.0,1.0,2.0,...,Partial,Caucasian,3.0,Other,None,"[3.351129363449692, 6.373716632443532, 13.1088...","['No', 'No', 'URTI', 'LRTI', 'URTI', 'URTI', '...","['No', 'No', 'Mild', 'Mild', 'Mild', 'Moderate...",0.0,0.0


,Gest_Days,Weight_0m,Weight_3m,Weight_12m,BF_Implied_Duration,PSS_36week,CSED_36week,PSS_18week,CSED_18week,PSS_6m,...,BF_Status_6m,Child_Ethinicity,Mode_of_delivery,Mother_Condition_Delivery,First_10min_Measure,Respiratory_Report_Months,Respiratory_Infections,Severity_of_Respiratoryinfections,Wheeze_3m,Noncold_Wheeze_3m
Subject_Number,,,,,,,,,,,,,,,,,,,,,
20059,276.0,3.346,6.20,NaN,12.0,10.0,14.0,13.0,7.0,16.0,...,Partial,Caucasian,Vaginal,2.0,0.0,"[nan, nan, nan, 3.0882956878850107, 6.24229979...","[nan, nan, nan, 'No', 'No', 'No', 'LRTI', 'LRT...","[nan, nan, nan, 'No', 'No', 'No', 'Mild', 'Mil...",0.0,0.0
20061,269.0,3.370,6.53,9.30,8.0,5.0,4.0,11.0,4.0,9.0,...,Partial,HalfCaucas,Caesarean,NaN,0.0,"[nan, nan, nan, 3.383983572895277, 6.340862422...","[nan, nan, nan, 'No', 'No', 'LRTI', 'LRTI', 'L...","[nan, nan, nan, 'No', 'No', 'Mild', 'Severe', ...",0.0,0.0
20062,279.0,3.055,5.60,8.18,6.0,15.0,15.0,10.0,3.0,18.0,...,Partial,Caucasian,Vaginal,0.0,NaN,"[nan, nan, nan, nan, 2.9897330595482545, 7.129...","[nan, nan, nan, nan, 'No', 'LRTI', 'LRTI', 'LR...","[nan, nan, nan, nan, 'No', 'Mild', 'Mild', 'Mi...",0.0,0.0
20063,273.0,3.485,5.90,NaN,11.0,5.0,3.0,4.0,1.0,8.0,...,Exclusive,Caucasian,Vaginal,0.0,0.0,"[nan, nan, nan, nan, 2.9568788501026693, 13.96...","[nan, nan, nan, nan, 'No', 'No', 'No', 'No', '...","[nan, nan, nan, nan, 'No', 'No', 'No', 'No', '...",0.0,0.0
20064,258.0,3.940,7.30,9.45,2.5,24.0,12.0,17.0,18.0,23.0,...,Zero,Caucasian,Caesarean,1.0,NaN,"[nan, 3.0882956878850107, 7.063655030800822, 1...","[nan, 'No', 'No', 'No', 'No', 'No', 'No', 'No'...","[nan, 'No', 'No', 'No', 'No', 'No', 'No', 'No'...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,280.0,3.800,5.52,8.50,14.0,26.0,39.0,12.0,8.0,12.0,...,Exclusive,Caucasian,Vaginal,1.0,0.0,"[nan, nan, 2.9897330595482545, 8.0821355236139...","[nan, nan, 'No', 'No', 'No', 'LRTI', 'LRTI', '...","[nan, nan, 'No', 'No', 'No', 'Mild', 'Mild', '...",0.0,0.0
50860,281.0,3.520,5.52,8.20,0.5,8.0,5.0,5.0,0.0,9.0,...,Zero,HalfCaucas,Vaginal,0.0,0.0,"[nan, nan, nan, nan, nan, 2.7597535934291586, ...","[nan, nan, nan, nan, nan, 'No', 'No', 'LRTI', ...","[nan, nan, nan, nan, nan, 'No', 'No', 'Mild', ...",0.0,0.0
50862,272.0,3.648,7.04,9.70,12.0,4.0,3.0,2.0,1.0,2.0,...,Partial,Caucasian,Vaginal,1.0,0.0,"[3.351129363449692, 6.373716632443532, 13.1088...","['No', 'No', 'URTI', 'LRTI', 'URTI', 'URTI', '...","['No', 'No', 'Mild', 'Mild', 'Mild', 'Moderate...",0.0,0.0


,Gest_Days,Weight_0m,Weight_3m,Weight_12m,BF_Implied_Duration,PSS_36week,CSED_36week,PSS_18week,CSED_18week,PSS_6m,...,BF_Status_6m,Child_Ethinicity,Mode_of_delivery,Mother_Condition_Delivery,First_10min_Measure,Respiratory_Report_Months,Respiratory_Infections,Severity_of_Respiratoryinfections,Wheeze_3m,Noncold_Wheeze_3m
Subject_Number,,,,,,,,,,,,,,,,,,,,,
20059,276.0,3.346,6.20,NaN,12.0,10.0,14.0,13.0,7.0,16.0,...,Partial,Caucasian,Vaginal,2.0,0.0,"[nan, nan, nan, 3.0882956878850107, 6.24229979...","[nan, nan, nan, 'No', 'No', 'No', 'LRTI', 'LRT...","[nan, nan, nan, 'No', 'No', 'No', 'Mild', 'Mil...",0.0,0.0
20061,269.0,3.370,6.53,9.30,8.0,5.0,4.0,11.0,4.0,9.0,...,Partial,HalfCaucas,Caesarean,NaN,0.0,"[nan, nan, nan, 3.383983572895277, 6.340862422...","[nan, nan, nan, 'No', 'No', 'LRTI', 'LRTI', 'L...","[nan, nan, nan, 'No', 'No', 'Mild', 'Severe', ...",0.0,0.0
20062,279.0,3.055,5.60,8.18,6.0,15.0,15.0,10.0,3.0,18.0,...,Partial,Caucasian,Vaginal,0.0,NaN,"[nan, nan, nan, nan, 2.9897330595482545, 7.129...","[nan, nan, nan, nan, 'No', 'LRTI', 'LRTI', 'LR...","[nan, nan, nan, nan, 'No', 'Mild', 'Mild', 'Mi...",0.0,0.0
20063,273.0,3.485,5.90,NaN,11.0,5.0,3.0,4.0,1.0,8.0,...,Exclusive,Caucasian,Vaginal,0.0,0.0,"[nan, nan, nan, nan, 2.9568788501026693, 13.96...","[nan, nan, nan, nan, 'No', 'No', 'No', 'No', '...","[nan, nan, nan, nan, 'No', 'No', 'No', 'No', '...",0.0,0.0
20064,258.0,3.940,7.30,9.45,2.5,24.0,12.0,17.0,18.0,23.0,...,Zero,Caucasian,Caesarean,1.0,NaN,"[nan, 3.0882956878850107, 7.063655030800822, 1...","[nan, 'No', 'No', 'No', 'No', 'No', 'No', 'No'...","[nan, 'No', 'No', 'No', 'No', 'No', 'No', 'No'...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,280.0,3.800,5.52,8.50,14.0,26.0,39.0,12.0,8.0,12.0,...,Exclusive,Caucasian,Vaginal,1.0,0.0,"[nan, nan, 2.9897330595482545, 8.0821355236139...","[nan, nan, 'No', 'No', 'No', 'LRTI', 'LRTI', '...","[nan, nan, 'No', 'No', 'No', 'Mild', 'Mild', '...",0.0,0.0
50860,281.0,3.520,5.52,8.20,0.5,8.0,5.0,5.0,0.0,9.0,...,Zero,HalfCaucas,Vaginal,0.0,0.0,"[nan, nan, nan, nan, nan, 2.7597535934291586, ...","[nan, nan, nan, nan, nan, 'No', 'No', 'LRTI', ...","[nan, nan, nan, nan, nan, 'No', 'No', 'Mild', ...",0.0,0.0
50862,272.0,3.648,7.04,9.70,12.0,4.0,3.0,2.0,1.0,2.0,...,Partial,Caucasian,Vaginal,1.0,0.0,"[3.351129363449692, 6.373716632443532, 13.1088...","['No', 'No', 'URTI', 'LRTI', 'URTI', 'URTI', '...","['No', 'No', 'Mild', 'Mild', 'Mild', 'Moderate...",0.0,0.0


The output columns will be: RIfrequency_earlier_report, RIseverity_earlier_report, RIfrequency_later_report, RIseverity_later_report


,Gest_Days,Weight_0m,Weight_3m,Weight_12m,BF_Implied_Duration,PSS_36week,CSED_36week,PSS_18week,CSED_18week,PSS_6m,...,Child_Ethinicity,Mode_of_delivery,Mother_Condition_Delivery,First_10min_Measure,Wheeze_3m,Noncold_Wheeze_3m,RIfrequency_earlier_report,RIfrequency_later_report,RIseverity_earlier_report,RIseverity_later_report
Subject_Number,,,,,,,,,,,,,,,,,,,,,
20059,276.0,3.346,6.20,NaN,12.0,10.0,14.0,13.0,7.0,16.0,...,Caucasian,Vaginal,2.0,0.0,0.0,0.0,0,3.0,0.0,0.3
20061,269.0,3.370,6.53,9.30,8.0,5.0,4.0,11.0,4.0,9.0,...,HalfCaucas,Caesarean,NaN,0.0,0.0,0.0,1,3.0,0.1,0.8
20062,279.0,3.055,5.60,8.18,6.0,15.0,15.0,10.0,3.0,18.0,...,Caucasian,Vaginal,0.0,NaN,0.0,0.0,1,2.0,0.1,0.3
20063,273.0,3.485,5.90,NaN,11.0,5.0,3.0,4.0,1.0,8.0,...,Caucasian,Vaginal,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
20064,258.0,3.940,7.30,9.45,2.5,24.0,12.0,17.0,18.0,23.0,...,Caucasian,Caesarean,1.0,NaN,0.0,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,280.0,3.800,5.52,8.50,14.0,26.0,39.0,12.0,8.0,12.0,...,Caucasian,Vaginal,1.0,0.0,0.0,0.0,0,3.0,0.0,0.3
50860,281.0,3.520,5.52,8.20,0.5,8.0,5.0,5.0,0.0,9.0,...,HalfCaucas,Vaginal,0.0,0.0,0.0,0.0,1,0.0,0.1,0.0
50862,272.0,3.648,7.04,9.70,12.0,4.0,3.0,2.0,1.0,2.0,...,Caucasian,Vaginal,1.0,0.0,0.0,0.0,1,2.0,0.2,0.8


...

,Gest_Days,Weight_0m,Weight_3m,Weight_12m,BF_Implied_Duration,PSS_36week,CSED_36week,PSS_18week,CSED_18week,PSS_6m,...,BF_Status_3m,BF_Status_6m,Child_Ethinicity,Mode_of_delivery,Mother_Condition_Delivery,Wheeze_3m,RIfrequency_earlier_report,RIfrequency_later_report,RIseverity_earlier_report,RIseverity_later_report
Subject_Number,,,,,,,,,,,,,,,,,,,,,
20059,276.0,3.346,6.20,NaN,12.0,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Moderate_Stress,...,Partial,Partial,Caucasian,Vaginal,2.0,0.0,0,3.0,0.0,0.3
20061,269.0,3.370,6.53,9.30,8.0,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Low_Stress,...,Exclusive,Partial,HalfCaucas,Caesarean,NaN,0.0,1,3.0,0.1,0.8
20062,279.0,3.055,5.60,8.18,6.0,Moderate_Stress,Low_Depression,Low_Stress,Low_Depression,Moderate_Stress,...,Exclusive,Partial,Caucasian,Vaginal,0.0,0.0,1,2.0,0.1,0.3
20063,273.0,3.485,5.90,NaN,11.0,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Low_Stress,...,Exclusive,Exclusive,Caucasian,Vaginal,0.0,0.0,0,0.0,0.0,0.0
20064,258.0,3.940,7.30,9.45,2.5,Moderate_Stress,Low_Depression,Moderate_Stress,Low_Depression,Moderate_Stress,...,Zero,Zero,Caucasian,Caesarean,1.0,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,280.0,3.800,5.52,8.50,14.0,Moderate_Stress,High_Depression,Low_Stress,Low_Depression,Low_Stress,...,Exclusive,Exclusive,Caucasian,Vaginal,1.0,0.0,0,3.0,0.0,0.3
50860,281.0,3.520,5.52,8.20,0.5,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Low_Stress,...,Zero,Zero,HalfCaucas,Vaginal,0.0,0.0,1,0.0,0.1,0.0
50862,272.0,3.648,7.04,9.70,12.0,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Low_Stress,...,Partial,Partial,Caucasian,Vaginal,1.0,0.0,1,2.0,0.2,0.8


All those with missing value will be imputated, the features with number of missing values are:  (['No_of_Pregnancy', 'Gest_Days', 'Prenatal_Second_Hand', 'Prenatal_Maternal_Smoke', 'Weight_0m', 'BF_Status_6m', 'Mother_Asthma', 'Apgar_Score_1min', 'Mom_Atopy', 'Apgar_Score_5min', 'Mom_Food', 'Wheeze_3m', 'RIfrequency_later_report', 'RIseverity_later_report', 'BF_9m', 'Weight_12m', 'BF_12m', 'Mother_Condition_Delivery', 'Respiratory_Problems_Birth', 'Anesthetic_delivery', 'Epi_Noncold_Wheeze_3m', 'Stay_Duration_Hospital', 'Weight_3m', 'CSED_36week', 'PSS_36week', 'Analgesics_usage_delivery', 'Complications_Birth', 'Jaundice_Birth', 'PSS_18week', 'CSED_18week', 'Home_New_Furnitures_6m', 'Home_Furry_Pets_6m', 'PSS_12m', 'Home_Presence_Smoke_6m', 'CSED_12m', 'PSS_6m', 'CSED_6m', 'Father_Asthma', 'Dad_Atopy', 'Dad_Food'], [2, 3, 12, 14, 19, 19, 19, 22, 22, 22, 25, 48, 51, 51, 56, 56, 56, 68, 78, 91, 93, 95, 103, 117, 124, 136, 146, 150, 166, 167, 271, 271, 274, 274, 274, 279, 284, 297, 425,

,Gest_Days,Weight_0m,Weight_3m,Weight_12m,BF_Implied_Duration,PSS_36week,CSED_36week,PSS_18week,CSED_18week,PSS_6m,...,BF_Status_3m,BF_Status_6m,Child_Ethinicity,Mode_of_delivery,Mother_Condition_Delivery,Wheeze_3m,RIfrequency_earlier_report,RIfrequency_later_report,RIseverity_earlier_report,RIseverity_later_report
Subject_Number,,,,,,,,,,,,,,,,,,,,,
20059,276.0,3.346,6.20,NaN,12.0,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Moderate_Stress,...,Partial,Partial,Caucasian,Vaginal,2.0,0.0,0,3.0,0.0,0.3
20061,269.0,3.370,6.53,9.30,8.0,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Low_Stress,...,Exclusive,Partial,HalfCaucas,Caesarean,NaN,0.0,1,3.0,0.1,0.8
20062,279.0,3.055,5.60,8.18,6.0,Moderate_Stress,Low_Depression,Low_Stress,Low_Depression,Moderate_Stress,...,Exclusive,Partial,Caucasian,Vaginal,0.0,0.0,1,2.0,0.1,0.3
20063,273.0,3.485,5.90,NaN,11.0,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Low_Stress,...,Exclusive,Exclusive,Caucasian,Vaginal,0.0,0.0,0,0.0,0.0,0.0
20064,258.0,3.940,7.30,9.45,2.5,Moderate_Stress,Low_Depression,Moderate_Stress,Low_Depression,Moderate_Stress,...,Zero,Zero,Caucasian,Caesarean,1.0,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,280.0,3.800,5.52,8.50,14.0,Moderate_Stress,High_Depression,Low_Stress,Low_Depression,Low_Stress,...,Exclusive,Exclusive,Caucasian,Vaginal,1.0,0.0,0,3.0,0.0,0.3
50860,281.0,3.520,5.52,8.20,0.5,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Low_Stress,...,Zero,Zero,HalfCaucas,Vaginal,0.0,0.0,1,0.0,0.1,0.0
50862,272.0,3.648,7.04,9.70,12.0,Low_Stress,Low_Depression,Low_Stress,Low_Depression,Low_Stress,...,Partial,Partial,Caucasian,Vaginal,1.0,0.0,1,2.0,0.2,0.8


,Gest_Days,Weight_0m,Weight_3m,Weight_12m,BF_Implied_Duration,No_of_Pregnancy,Stay_Duration_Hospital,Number_of_AntibioticsCourse,Epi_Noncold_Wheeze_3m,Apgar_Score_1min,...,BF_Status_3m_Exclusive,BF_Status_3m_Partial,BF_Status_3m_Zero,BF_Status_6m_Exclusive,BF_Status_6m_Partial,BF_Status_6m_Zero,Child_Ethinicity_Caucasian,Child_Ethinicity_HalfCaucas,Child_Ethinicity_NonCaucas,Mode_of_delivery_Vaginal
Subject_Number,,,,,,,,,,,,,,,,,,,,,
20059,276.0,3.346,6.20,NaN,12.0,1.609438,0.693147,0.000000,0.0,2.302585,...,0,1,0,0,1,0,1,0,0,1
20061,269.0,3.370,6.53,9.30,8.0,0.693147,1.386294,0.000000,0.0,2.302585,...,1,0,0,0,1,0,0,1,0,0
20062,279.0,3.055,5.60,8.18,6.0,1.098612,1.386294,0.000000,0.0,2.302585,...,1,0,0,0,1,0,1,0,0,1
20063,273.0,3.485,5.90,NaN,11.0,1.098612,NaN,0.000000,0.0,2.302585,...,1,0,0,1,0,0,1,0,0,1
20064,258.0,3.940,7.30,9.45,2.5,1.386294,1.609438,0.000000,0.0,2.197225,...,0,0,1,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,280.0,3.800,5.52,8.50,14.0,1.098612,0.693147,0.000000,0.0,2.302585,...,1,0,0,1,0,0,1,0,0,1
50860,281.0,3.520,5.52,8.20,0.5,0.693147,1.098612,0.693147,0.0,2.302585,...,0,0,1,0,0,1,0,1,0,1
50862,272.0,3.648,7.04,9.70,12.0,0.693147,1.098612,0.000000,0.0,2.302585,...,0,1,0,0,1,0,1,0,0,1


<IPython.core.display.Javascript object>

In [24]:
NumNaNimputer().fit_transform(X)

,Gest_Days,Weight_0m,Weight_3m,Weight_12m,BF_Implied_Duration,No_of_Pregnancy,Stay_Duration_Hospital,Number_of_AntibioticsCourse,Epi_Noncold_Wheeze_3m,Apgar_Score_1min,...,Home_Presence_Smoke_6m_Missing,Anesthetic_delivery_Missing,Dad_Atopy_Missing,Home_New_Furnitures_6m_Missing,Analgesics_usage_delivery_Missing,Complications_Birth_Missing,Father_Asthma_Missing,Home_Furry_Pets_6m_Missing,Mother_Condition_Delivery_Missing,Stay_Duration_Hospital_Missing
Subject_Number,,,,,,,,,,,,,,,,,,,,,
20059,276.0,3.346,6.20,9.34,12.0,1.609438,0.693147,0.000000,0.0,2.302585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20061,269.0,3.370,6.53,9.30,8.0,0.693147,1.386294,0.000000,0.0,2.302585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
20062,279.0,3.055,5.60,8.18,6.0,1.098612,1.386294,0.000000,0.0,2.302585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20063,273.0,3.485,5.90,10.18,11.0,1.098612,1.098612,0.000000,0.0,2.302585,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
20064,258.0,3.940,7.30,9.45,2.5,1.386294,1.609438,0.000000,0.0,2.197225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,280.0,3.800,5.52,8.50,14.0,1.098612,0.693147,0.000000,0.0,2.302585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50860,281.0,3.520,5.52,8.20,0.5,0.693147,1.098612,0.693147,0.0,2.302585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50862,272.0,3.648,7.04,9.70,12.0,0.693147,1.098612,0.000000,0.0,2.302585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [23]:
# Imputation on Numeric features in combination with imputation techniques (Simple, KNN, Missforest, etc)
class NumNaNimputer(BaseEstimator, TransformerMixin):
    """
    Impute Numeric columns based on the characteristics of the each feature -- number of unique values, number of nullness, and feature groups
    """

    def __init__(self, add_indicator_threshold=30, imputing_correlated_subset="KNN"):
        super().__init__()
        self.add_indicator_threshold = add_indicator_threshold
        self.imputing_correlated_subset = imputing_correlated_subset
        self.num_strategies = (
            {}
        )  # Dictionary to store columns for different kind of groups for imputing
        self.num_strategies_df_ = (
            pd.DataFrame()
        )  # DataFrame for overviewing the missingness for different columns
        # Imputer Initializing
        if self.imputing_correlated_subset == "KNN":
            self.BF_imputer = KNNImputer()
            self.Weight_imputer = KNNImputer()
            self.RESP_imputer = KNNImputer()
        else:
            self.BF_imputer = MissForest()
            self.Weight_imputer = MissForest()
            self.RESP_imputer = MissForest()
        self.Binary_ignore_missing = SimpleImputer(
            add_indicator=False, strategy="most_frequent"
        )
        self.Binary_indicator_missing = SimpleImputer(
            add_indicator=True, strategy="most_frequent"
        )
        self.Nonbinary_ignore_missing = SimpleImputer(
            add_indicator=False, strategy="median"
        )
        self.Nonbinary_indicator_missing = SimpleImputer(
            add_indicator=True, strategy="median"
        )

    def fit(self, X, y=None):
        """
        Generate numeric columns that will be processed differently based on the X characteristics
            Only those with a statistically significant missingness will be accompanied with a column of missingness indicator
        Fitted_outcome:
            1. Dictionary of different subsets of columns for different imputation schemes that will be referred during transformation.
                Valid keys include: BF_columns, Weight_columns, Resp_columns, Uni_binary_columns_indicator, Uni_binary_columns_ignore,
                Uni_nonbinary_columns_indicator_median, Uni_nonbinary_columns_ignore_median
            2. fitted imputer - using fit method
            3. Attribute: num_strategies_df_: DataFrame for overviewing the missingness for different columns
        """
        # Step 1: Strategize imputation techniques based on X feature characteristics
        cols_with_na = X.isna().sum()[X.isna().sum() > 0].index

        top_percentage = {}
        for i in X[cols_with_na].columns:
            top_percentage[i] = round(
                (X[i].value_counts(normalize=True).values[0] * 100), 2
            )
        per_ser = pd.Series(top_percentage)

        self.num_strategies_df_ = pd.concat(
            [X[cols_with_na].nunique(), X[cols_with_na].isna().sum(), per_ser], axis=1
        ).rename(
            columns={
                0: "Num_Unique_Values",
                1: "Num_Missing_Values",
                2: "Top_Percentage",
            }
        )

        self.num_strategies_df_.sort_values(
            by="Top_Percentage", ascending=False, inplace=True
        )

        # 1.1 For multi-variate imputation based on subset correlation: (KNN, Miss)

        # Based on pairwise correlation of columns,
        # KNN imputer for following subset of columns, the rest is more likely to be noise, rather than signal
        BF_columns = set(X.columns[X.columns.str.contains("BF_")])
        Weight_columns = set(X.columns[X.columns.str.contains("Weight_")]) | {
            "Gest_Days"
        }
        Resp_columns = set(X.columns[X.columns.str.contains("Wheeze_|RI|Respiratory")])

        self.num_strategies["BF_columns"] = BF_columns
        self.num_strategies["Weight_columns"] = Weight_columns
        self.num_strategies["Resp_columns"] = Resp_columns

        # 1.2 For Uni-Variate imputation

        # Simple for the rest columns with na values (Univariate)
        Ungrouped_columns = set(cols_with_na) - (
            BF_columns | Weight_columns | Resp_columns
        )
        Uni_binary_columns_indicator = set()
        Uni_binary_columns_ignore = set()
        Uni_nonbinary_columns_indicator_median = set()
        Uni_nonbinary_columns_ignore_median = set()

        for col in Ungrouped_columns:
            # Only mode - can be biased when there is no indicator, however, curse of dimensionality is another concern
            if (self.num_strategies_df_.loc[col]["Num_Unique_Values"] == 2) & (
                self.num_strategies_df_.loc[col]["Num_Missing_Values"]
                < self.add_indicator_threshold
            ):
                Uni_binary_columns_ignore.add(col)
            elif (self.num_strategies_df_.loc[col]["Num_Unique_Values"] == 2) & (
                self.num_strategies_df_.loc[col]["Num_Missing_Values"]
                >= self.add_indicator_threshold
            ):
                Uni_binary_columns_indicator.add(col)
            # Middle number will be accepted.
            elif (self.num_strategies_df_.loc[col]["Num_Unique_Values"] != 2) & (
                self.num_strategies_df_.loc[col]["Num_Missing_Values"]
                < self.add_indicator_threshold
            ):
                Uni_nonbinary_columns_ignore_median.add(col)
            else:
                Uni_nonbinary_columns_indicator_median.add(col)

        self.num_strategies[
            "Uni_binary_columns_indicator"
        ] = Uni_binary_columns_indicator
        self.num_strategies["Uni_binary_columns_ignore"] = Uni_binary_columns_ignore
        self.num_strategies[
            "Uni_nonbinary_columns_indicator_median"
        ] = Uni_nonbinary_columns_indicator_median
        self.num_strategies[
            "Uni_nonbinary_columns_ignore_median"
        ] = Uni_nonbinary_columns_ignore_median

        # Step 2: Begin fitting using different techniques

        # Multi-variate fit:
        self.BF_imputer.fit(X[self.num_strategies["BF_columns"]])
        self.Weight_imputer.fit(X[self.num_strategies["Weight_columns"]])
        self.RESP_imputer.fit(X[self.num_strategies["Resp_columns"]])

        # Uni-variate fit:
        self.Binary_ignore_missing.fit(
            X[self.num_strategies["Uni_binary_columns_ignore"]]
        )
        self.Binary_indicator_missing.fit(
            X[self.num_strategies["Uni_binary_columns_indicator"]]
        )
        self.Nonbinary_ignore_missing.fit(
            X[self.num_strategies["Uni_nonbinary_columns_ignore_median"]]
        )
        self.Nonbinary_indicator_missing.fit(
            X[self.num_strategies["Uni_nonbinary_columns_indicator_median"]]
        )

        return self

    def transform(self, X, y=None):

        # Multi-variate transform
        X.loc[:, self.num_strategies["BF_columns"]] = self.BF_imputer.transform(
            X[self.num_strategies["BF_columns"]]
        )
        X.loc[:, self.num_strategies["Weight_columns"]] = self.Weight_imputer.transform(
            X[self.num_strategies["Weight_columns"]]
        )
        X.loc[:, self.num_strategies["Resp_columns"]] = self.RESP_imputer.transform(
            X[self.num_strategies["Resp_columns"]]
        )

        # Uni-variate transform

        added_columns_binary = [
            i + "_Missing" for i in self.num_strategies["Uni_binary_columns_indicator"]
        ]
        added_columns_nonbinary = [
            i + "_Missing"
            for i in self.num_strategies["Uni_nonbinary_columns_indicator_median"]
        ]

        binary_columns_with_indicator = (
            list(self.num_strategies["Uni_binary_columns_indicator"])
            + added_columns_binary
        )
        nonbinary_columns_with_indicator = (
            list(self.num_strategies["Uni_nonbinary_columns_indicator_median"])
            + added_columns_nonbinary
        )

        # Begin imputation
        if len(self.num_strategies["Uni_binary_columns_indicator"]):
            X[binary_columns_with_indicator] = self.Binary_indicator_missing.transform(
                X[self.num_strategies["Uni_binary_columns_indicator"]]
            )

        if len(self.num_strategies["Uni_nonbinary_columns_indicator_median"]):
            X[
                nonbinary_columns_with_indicator
            ] = self.Nonbinary_indicator_missing.transform(
                X[self.num_strategies["Uni_nonbinary_columns_indicator_median"]]
            )

        if len(self.num_strategies["Uni_binary_columns_ignore"]):
            X[
                list(self.num_strategies["Uni_binary_columns_ignore"])
            ] = self.Binary_ignore_missing.transform(
                X[self.num_strategies["Uni_binary_columns_ignore"]]
            )

        if len(self.num_strategies["Uni_nonbinary_columns_ignore_median"]):
            X[
                list(self.num_strategies["Uni_nonbinary_columns_ignore_median"])
            ] = self.Nonbinary_ignore_missing.transform(
                X[self.num_strategies["Uni_nonbinary_columns_ignore_median"]]
            )

        return X

<IPython.core.display.Javascript object>

In [7]:
df_for_ml = generate_trainable_dataset(X, y, add_indicator_threshold=1000)

The output columns will be: RIfrequency_earlier_report, RIseverity_earlier_report, RIfrequency_later_report, RIseverity_later_report
---------------------------------------------------------------------------------------------------
Given the correlation threshhold of 0.9, the columns that will be removed are:['Antibiotics_Usage', 'Dad_Inhalant', 'Noncold_Wheeze_3m', 'Mom_Inhalant']. Please see the following correlation:{'Antibiotics_Usage <> Number_of_AntibioticsCourse': 0.938214467022626, 'Noncold_Wheeze_3m <> Epi_Noncold_Wheeze_3m': 0.931401423295512, 'Dad_Inhalant <> Dad_Atopy': 1.0, 'Mom_Inhalant <> Mom_Atopy': 0.9983089637136121}
---------------------------------------------------------------------------------------------------
Given the correlation threshhold of 0.7, the columns that will be considered to be dropped are:['First_10min_Measure', 'RIseverity_earlier_report', 'BF_12m', 'Mom_Inhalant', 'RIseverity_later_report', 'Parental_Asthma', 'BF_9m', 'Dad_Inhalant', 'Antibiotic

<IPython.core.display.Javascript object>

In [8]:
# Backup df_for_ml
df_for_ml_backup = df_for_ml.copy()

<IPython.core.display.Javascript object>

In [9]:
df_for_ml.y.value_counts(normalize=True)
df_for_ml.y.value_counts(normalize=False)
# Restore X,y for ML modelling
X = df_for_ml.drop(columns="y")
y = df_for_ml.y
X.shape
y.shape

0.0    0.844082
1.0    0.155918
Name: y, dtype: float64

0.0    2068
1.0     382
Name: y, dtype: int64

(2450, 99)

(2450,)

<IPython.core.display.Javascript object>

In [ ]:
# Scaled ML using MinMaxScaler()
df_ml_scaled = pd.DataFrame(
    MinMaxScaler().fit_transform(df_for_ml), columns=df_for_ml.columns
)

for i in df_ml_scaled.columns:
    df_ml_scaled[i] = df_ml_scaled[i].astype("float16")

X = df_ml_scaled.drop(columns="y")
y = df_ml_scaled.y
X.shape
y.shape

In [10]:
corr_X_df = X.corr()
repetition_drop = []  # Feature will be dropped because of high correlation
repetition_dict = {}

for i, col in enumerate(corr_X_df.columns):
    for i_name in corr_X_df[col].index[i + 1 :]:
        if corr_X_df[col][i_name] > 0.95:
            repetition_drop.extend([i_name])
            repetition_dict[" <> ".join([i_name, corr_X_df[col].name])] = corr_X_df[
                col
            ][i_name]

repetition_drop
repetition_dict

['CSED_36week_Missing',
 'CSED_18week_Missing',
 'CSED_6m_Missing',
 'CSED_12m_Missing']

{'CSED_36week_Missing <> PSS_36week_Missing': 0.9699058395018689,
 'CSED_18week_Missing <> PSS_18week_Missing': 0.9967832108354368,
 'CSED_6m_Missing <> PSS_6m_Missing': 0.9900160742901514,
 'CSED_12m_Missing <> PSS_12m_Missing': 1.0}

<IPython.core.display.Javascript object>

## Now begins the INTERESTING ML Journey!

In [ ]:
def view_module_functions(module_alias):
    """
    Quick review of self-written functions and classes
    """
    return [
        i
        for i in dir(module_alias)
        if (not i.isupper()) & ~(i.startswith("_")) & (len(i) > 6)
    ]


print(view_module_functions(UT))
print(view_module_functions(DT))

In [ ]:
# Summary of created dataframe for CHILD
def df_summary(X):
    """
    Overview the number of unique values, Top percentage of value distribution, for each feature 
    """

    cols_to_inspect = X.columns

    top_percentage = {}
    for i in X[cols_to_inspect].columns:
        top_percentage[i] = round(
            (X[i].value_counts(normalize=True).values[0] * 100), 2
        )
    per_ser = pd.Series(top_percentage)

    df_overview = (
        pd.concat(
            [
                X[cols_to_inspect].nunique(),
                X[cols_to_inspect].mean(),
                per_ser,
                X[cols_to_inspect].max(),
                X[cols_to_inspect].sum(),
                X[cols_to_inspect].var(),
            ],
            axis=1,
        )
        .rename(
            columns={
                0: "Num_Unique_Values",
                1: "Mean_Value",
                2: "Top_Percentage",
                3: "Max_Value",
                4: "Number_of_Binary_Positive",
                5: "Variance",
            }
        )
        .sort_values(by="Top_Percentage", ascending=False)
    )

    return df_overview


X_summary = df_summary(X)
X_summary.sort_values(by="Variance")

In [ ]:
def view_y_proportions(df, columns_of_interest, thresh=0):
    """
    Gadget to view the asthma proportions for different columns during feature selection
    """
    general_proportion = round((100 * df[df.y == 1].shape[0] / df.shape[0]), 2)
    print(
        "The proportion of definite asthma outcome for general cohorts is {}%".format(
            general_proportion
        )
    )
    asthma_for_col_number = []
    asthma_for_col_proportion = []
    col_with_thresh_number = []
    for i in columns_of_interest:

        total_of_subsamples = df[df[i] > thresh].shape[0]

        if 1 in df[df[i] > thresh].y.value_counts().sort_values(ascending=True).index:
            number_of_asthma = (
                df[df[i] > thresh].y.value_counts().sort_values(ascending=True).loc[1]
            )
        else:
            number_of_asthma = 0

        proportion = round(
            (number_of_asthma / (total_of_subsamples + 0.000001) * 100), 2
        )

        col_with_thresh_number.append(total_of_subsamples)
        asthma_for_col_proportion.append(proportion)
        asthma_for_col_number.append(number_of_asthma)
    #         print(
    #             "The proportion of asthma outcome when the value of {} is greater than {} is {}% or {} out of {}".format(
    #                 i, thresh, proportion, number_of_asthma, total_of_subsamples
    #             )
    #         )

    col_y_df = pd.DataFrame(
        [
            asthma_for_col_proportion,
            asthma_for_col_number,
            col_with_thresh_number,
            np.ones(len(col_with_thresh_number)) * thresh,
        ],
        index=[
            "Asthma_Proportion_over_thresh",
            "Asthma_Outcome_over_thresh",
            "Total_Number_over_thresh",
            "Thresh",
        ],
        columns=columns_of_interest,
    )

    general_ser = pd.Series(
        [general_proportion, df[df.y == 1].shape[0], df.shape[0], 0],
        name="*****GENERAL*****",
        index=[
            "Asthma_Proportion_over_thresh",
            "Asthma_Outcome_over_thresh",
            "Total_Number_over_thresh",
            "Thresh",
        ],
    )

    return col_y_df.T.append(general_ser).sort_values(
        by="Asthma_Proportion_over_thresh", ascending=False
    )


y_prop = view_y_proportions(df_ml_scaled, df_ml_scaled.columns[:-1], 0)

#### view_y_proportions(df_ml_scaled, df_ml_scaled.columns[:-1], 0.5)[
    view_y_proportions(
        df_ml_scaled, df_ml_scaled.columns[:-1], 0.5
    ).Asthma_Proportion_over_thresh
    > 30
]

view_y_proportions(df_ml_scaled, df_ml_scaled.columns[:-1], 0.5)[
    view_y_proportions(
        df_ml_scaled, df_ml_scaled.columns[:-1], 0.5
    ).Asthma_Proportion_over_thresh
    < 10
]

In [ ]:
y_prop

In [ ]:
# pd.concat([X_summary, y_prop], axis=1)

#### Additional: Statistic Analysis for F10min Mask related to Asthma Outcome

Pingouin, statsmodel(table,proportion), scipy, sklearn

In [ ]:
# Three Package to obtain P value for chi square test for F10min Oxygen Mask Stats
# Scipy Based
# One
import pingouin as pg

expected, observed, stats = pg.chi2_independence(
    df_ml_scaled, x="F10min_Oxygen_Mask", y="y", correction=False
)

observed, expected, stats

_, _, stats = pg.chi2_independence(
    df_ml_scaled, x="Prenatal_Hypotension", y="y", correction=False
)

stats

In [ ]:
# Two: Scipy - Required to create contingency table as input
# Need to create a contingency table first, which can be realized using pd.crosstab

from scipy.stats import chi2_contingency

contingency_f10m_Oxy = pd.crosstab(df_ml_scaled.F10min_Oxygen_Mask, df_ml_scaled.y)

chi2_contingency(contingency_f10m_Oxy.values, correction=False)

print(
    "In statistics, Yates's correction for continuity (or Yates's chi-squared test) is used in certain situations when testing for independence in a contingency table. It aims at correcting the error introduced by assuming that the discrete probabilities of frequencies in the table can be approximated by a continuous distribution (chi-squared). In some cases, Yates's correction may adjust too far, and so its current use is limited."
)

In [ ]:
# Three: Statsmodel
import statsmodels.api as sm

# sm_f10oxy_table = sm.stats.Table(contingency_f10m_Oxy.values)
sm_f10oxy_table = sm.stats.Table.from_data(df_ml_scaled[["F10min_Oxygen_Mask", "y"]])

print(f"Observed Table:\n {sm_f10oxy_table.table_orig}")
print(f"Expected Table:\n {sm_f10oxy_table.fittedvalues}")
print(
    f"Pearson Residuals:\n {sm_f10oxy_table.resid_pearson}"
)  # view Residuals which identify particular cells that most strongly violate independence

print("\n----------------Independence/Association test----------------")
print("\nNominal Association:\n", sm_f10oxy_table.test_nominal_association())
print("\nOrdinal Association:\n", sm_f10oxy_table.test_ordinal_association())
print(f"\nChi Square Contribution:\n{sm_f10oxy_table.chi2_contribs}")
print(
    "\n----------------Local Odd Ratio----------------:\n{}".format(
        sm_f10oxy_table.local_oddsratios
    )
)
print(
    f"Contingency table odd ratio: {sm.stats.Table2x2(contingency_f10m_Oxy.values).oddsratio}"
)


# Three-2: Statsmodel Proportion Test
# Proportion Chi-Square Test
print("\n----------------Proportion Chi-Square Test----------------")
from statsmodels.stats.proportion import (
    proportions_chisquare,
    proportion_effectsize,
)  # No indexing needed

f10mask_no_group_observation = np.sum(df_ml_scaled.F10min_Oxygen_Mask != 1)
f10mask_yes_group_observation = np.sum(df_ml_scaled.F10min_Oxygen_Mask == 1)

f10mask_no_asthma_count = np.sum(df_ml_scaled[df_ml_scaled.F10min_Oxygen_Mask != 1].y)
f10mask_yes_asthma_count = np.sum(df_ml_scaled[df_ml_scaled.F10min_Oxygen_Mask == 1].y)

(chi2, chi2_p_value, expected) = proportions_chisquare(
    count=[f10mask_no_asthma_count, f10mask_yes_asthma_count],
    nobs=[f10mask_no_group_observation, f10mask_yes_group_observation],
)

print(
    "\nAsthma Outcome Proportion Chi Square Test:\nchi2: %f \t p_value: %f"
    % (chi2, chi2_p_value)
)


asthma_no_group_observation = np.sum(df_ml_scaled.y != 1)
asthma_yes_group_observation = np.sum(df_ml_scaled.y == 1)

asthma_no_f10mask_count = np.sum(df_ml_scaled[df_ml_scaled.y != 1].F10min_Oxygen_Mask)
asthma_yes_f10mask_count = np.sum(df_ml_scaled[df_ml_scaled.y == 1].F10min_Oxygen_Mask)

(chi2, chi2_p_value, expected) = proportions_chisquare(
    count=[asthma_no_f10mask_count, asthma_yes_f10mask_count],
    nobs=[asthma_no_group_observation, asthma_yes_group_observation],
)

print(
    "\nF10m Mask Outcome Proportion Chisquare Test:\nchi2: %f \t p_value: %f"
    % (chi2, chi2_p_value)
)

In [ ]:
print("Discrepancy: P-value of chi square test between Scipy and Statsmodel")

print(
    f"\nFor Scipy with correction:\npvalue: {chi2_contingency(contingency_f10m_Oxy.values)[1]}"
)
print(
    f"\nFor Scipy without correction:\npvalue: {chi2_contingency(contingency_f10m_Oxy.values, correction=False)[1]}"
)

print(f"\nFor Statsmodel:\n{sm_f10oxy_table.test_nominal_association()}")


print(
    "\nExplanation:\n\nScipy uses the continuity correction, Statsmodels does not. If you pass correction=False to the scipy test, then the results will be identical."
)

In [ ]:
# Four: Sklearn - CHI2
from sklearn.feature_selection import chi2

chi2(df_ml_scaled.F10min_Mask_Ventilation.values.reshape(-1, 1), df_ml_scaled.y)

### ML Libraries

In [ ]:
# Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# Feature selection
from sklearn.feature_selection import (
    SelectPercentile,
    SelectKBest,
    SelectFwe,
    SelectFpr,
    SelectFromModel,
    chi2,
    mutual_info_classif,
    f_classif,
    SequentialFeatureSelector,
    RFECV,
)
from sklearn.inspection import permutation_importance

# Imbalance Learn
from imblearn.over_sampling import RandomOverSampler

# Models
from sklearn.linear_model import LogisticRegression, Lasso, LassoLarsCV, LassoCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Ensemble
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Model Metrics, Parameters, Evaluation
from sklearn.model_selection import (
    StratifiedKFold,
    GridSearchCV,
    cross_val_score,
    train_test_split,
)


from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    roc_curve,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)

# Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

### Feature Selection   <a id='0'></a> 

1. [Lasso](#lasso)
2. [Chi2](#chi2)
3. [f_value](#f_classif)
4. [FalsePositiveRate](#fpr)
5. [p_value](#fwe)
6. [mutual_information](#mutal)
7. [Recursive Feature Elimination](#RFE)
8. [SelectFromModel](#selm)
9. [permutation_importance](#permutation)
10. [SequentialFeatureSelector](#sequentialgreed)

### Feature Engineering
1. [PCA](#pca)
2. [FeatureAgglomeration](#clusteragglomeration)
3. [Derived_New_Features_with_Expertise_and_Logic](#derivenewfeature)

#### (1) Lasso - Observation - 1

In [ ]:
# plt.style.available
# plt.style.use("seaborn-bright")

In [ ]:
# Automatic Alphas with StratifiedKfold
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=1015)
model = LassoLarsCV(cv=skf).fit(X, y)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 12))
columns_of_total = X.shape[1]
start = 0
end = X.shape[1]
number_of_display = end - start

cm = iter(plt.get_cmap("Set1")(np.linspace(0, 1, number_of_display)))
for i in range(start, end):
    c = next(cm)
    _ = ax.plot(
        model.alphas_, model.coef_path_.T[:, i], c=c, alpha=0.8, label=X.columns[i]
    )


lassocv_df = pd.DataFrame(
    data=model.coef_path_.T, columns=X.columns, index=model.alphas_
)

y_pos_ser = lassocv_df.iloc[-1:].T.iloc[:, 0][lassocv_df.iloc[-1:].T.iloc[:, 0] != 0]
x_pos = float(lassocv_df.iloc[-1:].T.columns.values)
x_pos_list = [x_pos for i in range(len(y_pos_ser))]

for x_t, y_t, text in zip(x_pos_list, y_pos_ser.values, y_pos_ser.index):
    axtxt = ax.text(
        x_t - 0.00005,
        y_t,
        text,  # Used to format it K representation
        color="black",
        rotation="horizontal",
        size="large",
    )

ax.legend(X.columns[start:end], bbox_to_anchor=(1, 1))

plt.xlim([0.00015, 0.00085])
plt.ylabel("Lasso Regression CV")
plt.xlabel("Regularization Factor - Alphas")
plt.title("Regression Coefficient Progression for Lasso Path")

fig.savefig("../images/lasso_progression",dpi=300,bbox_inches='tight')


In [ ]:
view_y_proportions(
    df_for_ml, lassocv_df.sum()[lassocv_df.sum() != 0].sort_values().index, 0.5
)

#### Lasso - Observation - 2 - All features decreasing pattern   <a id='lasso'></a>

[return](#0)

In [ ]:
coef_dict = {}
for alp in tqdm(np.arange(0.000001, 0.0009, 0.000005)):
    lasso_model = Lasso(alpha=alp).fit(X, y)
    coef_dict[alp] = list(lasso_model.coef_)

res = pd.DataFrame(data=coef_dict.values(), columns=X.columns, index=coef_dict.keys()).T

In [ ]:
# For those with bitter coefficient (impact)
res[0.000001][abs(res[0.000001]) > 0.05].sort_values().index
view_y_proportions

In [ ]:
# sns.color_palette("YlOrBr", as_cmap=True)
fig, ax = plt.subplots(figsize=(40, 40))
sns.heatmap(res.iloc[:, :], vmax=0.2, vmin=-0.2, cmap="vlag", ax=ax)

#### (2) Chi2 & SelectPercentile  <a id='chi2'></a>

[return](#0)

In [ ]:
clf = Pipeline([("chi2", SelectPercentile(chi2)), ("rfc", RandomForestClassifier())])

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1020)

In [ ]:
# #############################################################################
# Plot the cross-validation score as a function of percentile of features
score_means = list()
score_stds = list()
percentiles = (1, 5, 10, 15, 20, 30, 40, 60, 80, 90, 100)

for percentile in tqdm(percentiles):
    clf.set_params(chi2__percentile=percentile)
    this_scores = cross_val_score(clf, X, y, cv=skf, scoring="precision")
    score_means.append(this_scores.mean())
    score_stds.append(this_scores.std())


plt.figure(figsize=(12, 8))
plt.errorbar(percentiles, score_means, np.array(score_stds))
plt.title(
    "Performance of the RandomForest-Chi2 varying the percentile of features selected"
)
plt.xticks(np.linspace(0, 100, 11, endpoint=True))
plt.xlabel("Percentile")
plt.ylabel("Precision Score")
plt.axis("tight")
plt.show()

In [ ]:
chi_perc = SelectPercentile(chi2, percentile=30)
chi_perc.fit_transform(X, y)

In [ ]:
# X.columns[chi_perc.get_support()]
chi_perc.get_feature_names_out()
len(chi_perc.get_feature_names_out())

chi2_res = pd.DataFrame(
    [list(chi_perc.pvalues_), list(chi_perc.scores_)],
    index=["P_value", "Scores"],
    columns=X.columns,
).T

chi2_res[chi2_res.P_value < 0.05]

In [ ]:
view_y_proportions(df_ml_scaled, chi2_res[chi2_res.P_value < 0.05].index, 0)

#### (3) F_classif & SelectPercentile  <a id='f_classif'></a>

[return](#0)

In [ ]:
clf = Pipeline([("fvalue", SelectKBest(f_classif)), ("xgb", XGBClassifier())])

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1020)

In [ ]:
# #############################################################################
# Plot the cross-validation score as a function of percentile of features
score_means = list()
score_stds = list()
k_best = (10, 20, 25, 30, 35, 40, 50, 60, 70, 90, "all")

for i in tqdm(k_best):
    clf.set_params(fvalue__k=i)
    this_scores = cross_val_score(clf, X, y, cv=skf, scoring="f1")
    score_means.append(this_scores.mean())
    score_stds.append(this_scores.std())


plt.figure(figsize=(12, 8))
plt.errorbar(percentiles, score_means, np.array(score_stds))
plt.title("Performance of the XGB-f_value varying the number of features selected")
plt.xticks(np.linspace(0, 100, 11, endpoint=True))
plt.xlabel("Percentile")
plt.ylabel("Precision Score")
plt.axis("tight")
plt.show()

In [ ]:
f_bestk = SelectKBest(f_classif, k=40)
f_bestk.fit_transform(X, y)

f_bestk.get_feature_names_out()
len(f_bestk.get_feature_names_out())

f_res = pd.DataFrame(
    [list(chi_perc.pvalues_), list(chi_perc.scores_)],
    index=["P_value", "Scores"],
    columns=X.columns,
).T

f_res[f_res.P_value < 0.05]

f_res[f_res.Scores > 5]

#### (4) False Positive Rate  <a id='fpr'></a>

[return](#0)

In [ ]:
fpr_res = SelectFpr(score_func=chi2, alpha=0.05)
fpr_res.fit_transform(X, y).shape
col_temp = pd.DataFrame(fpr_res.pvalues_.reshape(1,len(X.columns)),columns=X.columns).T.loc[:,0][pd.DataFrame(fpr_res.pvalues_.reshape(1,len(X.columns)),columns=X.columns).T.loc[:,0] <= 0.05].index


In [ ]:
d1 = fpr_res.fit_transform(X, y)
d1

In [ ]:
d2 = X[col_temp].values
d2

In [ ]:
np.sum(d1 != d2)

#### (5) P_value  <a id='fwe'></a>

Use P-value instead of score to select features

[return](#0)

In [ ]:
d3 = SelectFwe(chi2, alpha=0.05).fit_transform(X, y)

In [ ]:
np.sum(d1 != d3)

#### (6) Mutual Information  <a id='mutal'></a>

Use Mutual Information to select features for classification

[return](#0)

In [ ]:
mic = mutual_info_classif(X, y, random_state=1)
fig, ax = plt.subplots(figsize=(10, 20))
mic_df = (
    pd.DataFrame({"feature": X.columns, "vimp": mic})
    .set_index("feature")
    .sort_values(by="vimp", ascending=True)
)
mic_df[mic_df.vimp != 0].shape[0]
mic_df[mic_df.vimp != 0].plot.barh(ax=ax)
mic_df[mic_df.vimp != 0].index.values

In [ ]:
view_y_proportions(df_ml_scaled, mic_df[mic_df.vimp != 0].index.values, 0.2)[:10]

#### (7) Recursive Feature Elimination  <a id='RFE'></a>

Use Recursive Feature Elimination to select features for classification

[return](#0)

In [ ]:
from yellowbrick.model_selection import RFECV

res_index = res[0.000001][abs(res[0.000001]) > 0.05].sort_values().index

lasso_index = lassocv_df.sum()[lassocv_df.sum() != 0].index

In [ ]:
list(set(list(res_index) + list(lasso_index)))

In [ ]:
cv = StratifiedKFold(2)
# visualizer = RFECV(RandomForestClassifier(), cv=cv, scoring="average_precision")
# visualizer1 = RFECV(RandomForestClassifier(), cv=cv, scoring="precision_weighted")
visualizer1 = RFECV(RandomForestClassifier(), cv=cv, scoring="roc_auc")
visualizer1.fit(
    X[list(set(list(res_index) + list(lasso_index)))], y
)  # Fit the data to the visualizer
visualizer1.show()  # Finalize and render the figure

In [ ]:
view_y_proportions(df_ml_scaled, list(set(list(res_index) + list(lasso_index))))

In [ ]:
pd.DataFrame(
    data=visualizer1.ranking_,
    index=X[list(set(list(res_index) + list(lasso_index)))].columns,
    columns=["ranking"],
).sort_values(by="ranking").T

In [ ]:
set_all = X.columns.to_list()

In [ ]:
cv = StratifiedKFold(2)
# visualizer = RFECV(RandomForestClassifier(), cv=cv, scoring="average_precision")
# visualizer1 = RFECV(RandomForestClassifier(), cv=cv, scoring="precision_weighted")
visualizer2 = RFECV(RandomForestClassifier(), cv=cv, scoring="roc_auc")
visualizer2.fit(X[set_to_see], y)  # Fit the data to the visualizer
visualizer2.show()  # Finalize and render the figure

In [ ]:
pd.DataFrame(
    data=visualizer2.ranking_, index=X[set_to_see].columns, columns=["ranking"],
).sort_values(by="ranking").T

#### (8) SelectFromModel <a id='selm'></a>

Use Recursive Feature Elimination to select features for classification

[return](#0)

In [ ]:
lasso = LassoCV(cv=skf).fit(X, y)
importance = np.abs(lasso.coef_)
feature_names = np.array(X.columns)
plt.bar(height=importance, x=feature_names)
plt.title("Feature importances via coefficients")
plt.show()

In [ ]:
sfm = SelectFromModel(estimator=LassoCV(cv=skf))

In [ ]:
sfm.fit(X, y)

In [ ]:
sfm.get_feature_names_out()

In [ ]:
X.columns[sfm.get_support()]
sfm.estimator_.coef_

# coef_ser = pd.Series(sfm.estimator_.coef_.reshape(99,), index=X.columns, name="coef")[
#     abs(coef_ser) > 0
# ]
# coef_ser

importance = np.abs(coef_ser.values)
feature_names = np.array(coef_ser.index)
plt.bar(height=importance, x=feature_names)
plt.title("Feature importances via coefficients")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X[X.columns[sfm.get_support()]], y, test_size=0.33, stratify=y, random_state=43
# )

# clf_knn = KNeighborsClassifier(n_neighbors=3, weights="distance")
# clf_knn.fit(X_train, y_train)
# y_pred = clf_knn.predict(X_test)
# confusion_matrix(y_test, y_pred)
# print(classification_report(y_test, y_pred))
# f1_score(y_test, y_pred)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X[X.columns[sfm.get_support()]], y, test_size=0.33, stratify=y, random_state=42
# )

# clf_lr = LogisticRegression()
# clf_lr.fit(X_train, y_train)
# y_pred = clf_lr.predict(X_test)
# confusion_matrix(y_test, y_pred)
# print(classification_report(y_test, y_pred))
# f1_score(y_test, y_pred)

In [ ]:
sfm_rf = SelectFromModel(estimator=RandomForestClassifier(), threshold="mean")
sfm_rf.fit(X, y)

In [ ]:
sfm_rf.get_feature_names_out()

In [ ]:
rf_ser = pd.Series(
    sfm_rf.estimator.feature_importances_.reshape(99,),
    index=X.columns,
    name="feature_importance",
)
rf_ser.sort_values(ascending=False)

#### (9) Permutation Importance <a id='permutation'></a>
[return](#0)

Permutation feature importance is a model inspection technique that can be used for any fitted estimator when the data is tabular. This is especially useful for non-linear or opaque estimators. The permutation feature importance is defined to be the decrease in a model score when a single feature value is randomly shuffled. This procedure breaks the relationship between the feature and the target, thus the drop in the model score is indicative of how much the model depends on the feature. This technique benefits from being model agnostic and can be calculated many times with different permutations of the feature.

In [ ]:
def randomsubset_permutation_importance(*, clf: object, percentile_of_features: float):
    """
    As when all features are included, none of the feature will have any importance, I therefore created this function 
    to view the feature importance of a random subset of features.
    : para: percentile_of_features
    : para: clf: a classification algorithm
    : return: a visualization of feature importance for current subset of features
    """
    number_of_features = int(len(X.columns.values) * percentile_of_features)
    selected_columns = random.sample(list(X.columns.values), number_of_features)
    clf.fit(X[selected_columns], y)
    result = permutation_importance(
        clf, X[selected_columns], y, n_repeats=10, random_state=1012, scoring="roc_auc",
    )
    perm_sorted_idx = result.importances_mean.argsort()
    plt.figure(figsize=(20, 10))
    plt.barh(
        width=result.importances_mean[perm_sorted_idx].T, y=X.columns[perm_sorted_idx],
    )
    r = result
    for i in r.importances_mean.argsort()[::-1]:
        if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            print(
                f"{X.columns[i]:<8}"
                f"{r.importances_mean[i]:.3f}"
                f" +/- {r.importances_std[i]:.3f}"
            )

In [ ]:
randomsubset_permutation_importance(
    clf=RandomForestClassifier(), percentile_of_features=0.3,
)

In [ ]:
df_shrunk[df_shrunk.Home_Furry_Pets_6m.isna()][
    df_shrunk.columns[df_shrunk.columns.str.contains("Home|Asthma|^y$")]
]

### ML Pipeline Test Run